In [22]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama3-70b-8192" 
)

response = llm.invoke("Who was the first person on the moon?")
print(response.content)

The first person to set foot on the moon was Neil Armstrong. He stepped out of the lunar module Eagle onto the moon's surface on July 20, 1969, during the Apollo 11 mission.

Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the moon.

Armstrong was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the moon during the mission. Michael Collins remained in orbit around the moon in the command module Columbia.

Armstrong's historic moonwalk lasted about two and a half hours, during which he and Aldrin collected samples and conducted experiments. Their mission marked a major milestone in space exploration and paved the way for future human spaceflight.


In [6]:
import chromadb
client = chromadb.Client()
collection = client.create_collection(name= 'collection')


In [7]:
collection.add(
    documents= [
        'This document is about New York',
        'This document is about Delhi'],
    ids= ['id1', 'id2']  )
    

/Users/karthiksarma/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%


In [14]:
%pip install -qU langchain_community beautifulsoup4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
all_docs= collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This document is about New York',
  'This document is about Delhi'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None]}

In [11]:
results =collection.query(
    query_texts= ['Query is about pollution'], n_results=2)
results

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This document is about New York']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.498237133026123, 1.6412968635559082]]}

In [13]:
collection.delete(ids= all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': []}

In [16]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.nike.com/lead-data-scientist/job/R-62512")
page_data= loader.load().pop().page_content
print(page_data)






















Lead Data Scientist










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select

In [18]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
) 
    

In [23]:
chain_extract= prompt_extract | llm
res= chain_extract.invoke(input= {'page_data':page_data})
print(res.content)


[
    {
        "role": "Lead Data Scientist",
        "experience": "4 years",
        "skills": ["Python", "Machine Learning", "Statistics", "Business Intelligence", "Data Analytics", "SQL programming", "Data Visualization", "Time Series Forecasting"]],
        "description": "Design, develop, and program methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources to generate actionable insights and solutions for client services and product enhancement; build products for analysis; interact with product and service teams to identify questions and issues for data analysis and experiments; develop and code software programs, algorithms and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources; identify meaningful insights from large data and metadata sources; interpret and communicate insights and findings from analysis and experiments to product, service, and business managers; lead others to solve

In [35]:
print(res.content)


[
    {
        "role": "Lead Data Scientist",
        "experience": "4 years",
        "skills": ["Python", "Machine Learning", "Statistics", "Business Intelligence", "Data Analytics", "SQL programming", "Data Visualization", "Time Series Forecasting"]],
        "description": "Design, develop, and program methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources to generate actionable insights and solutions for client services and product enhancement; build products for analysis; interact with product and service teams to identify questions and issues for data analysis and experiments; develop and code software programs, algorithms and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources; identify meaningful insights from large data and metadata sources; interpret and communicate insights and findings from analysis and experiments to product, service, and business managers; lead others to solve

In [37]:
from langchain_core.output_parsers import JsonOutputParser
json_parse= JsonOutputParser()
jss= json_parse.parse(res.content)
jss

In [39]:
import json

try:
    fixed = res.content.replace(']]', ']')  # Quick fix for common bracket issues
    parsed = json.loads(fixed)
    print(parsed)
except json.JSONDecodeError as e:
    print("JSON parsing error:", e)


[{'role': 'Lead Data Scientist', 'experience': '4 years', 'skills': ['Python', 'Machine Learning', 'Statistics', 'Business Intelligence', 'Data Analytics', 'SQL programming', 'Data Visualization', 'Time Series Forecasting'], 'description': 'Design, develop, and program methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources to generate actionable insights and solutions for client services and product enhancement; build products for analysis; interact with product and service teams to identify questions and issues for data analysis and experiments; develop and code software programs, algorithms and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources; identify meaningful insights from large data and metadata sources; interpret and communicate insights and findings from analysis and experiments to product, service, and business managers; lead others to solve complex problems; and anticipate custo

In [41]:
from langchain_core.output_parsers import JsonOutputParser
json_parse= JsonOutputParser()
jss = res.content  # ✅ already parsed
jss

'[\n    {\n        "role": "Lead Data Scientist",\n        "experience": "4 years",\n        "skills": ["Python", "Machine Learning", "Statistics", "Business Intelligence", "Data Analytics", "SQL programming", "Data Visualization", "Time Series Forecasting"]],\n        "description": "Design, develop, and program methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources to generate actionable insights and solutions for client services and product enhancement; build products for analysis; interact with product and service teams to identify questions and issues for data analysis and experiments; develop and code software programs, algorithms and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources; identify meaningful insights from large data and metadata sources; interpret and communicate insights and findings from analysis and experiments to product, service, and business managers; lead others to

In [42]:
fixed_str = res.content.replace('"]]', '"]')  # removes the extra ]
data = json.loads(fixed_str)


In [43]:
from langchain_core.output_parsers import JsonOutputParser

# You don't need to parse again if already done
# But if you're using parser directly:
parser = JsonOutputParser()
structured = parser.parse(json.dumps(data))  # ensure it's clean JSON string

# Now use structured as input to another chain/tool



In [48]:
type(structured) # Should print <class 'dict'>


dict

In [50]:
import pandas as pd
df= pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [54]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
col = client.get_or_create_collection(name="portfolio")

if not col.count():
    for _, row in df.iterrows():
        col.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [59]:
links= col.query(query_texts= job['skills'], n_results=2).get('metadatas' ,[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [58]:
job = structured
job['skills']

['Python',
 'Machine Learning',
 'Statistics',
 'Business Intelligence',
 'Data Analytics',
 'SQL programming',
 'Data Visualization',
 'Time Series Forecasting']

In [62]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Science Solutions for Your Business

Dear [Hiring Manager],

I came across your job posting for a Lead Data Scientist, and I'm excited to introduce you to AtliQ, a renowned AI & Software Consulting firm. Our team of experts can help you design, develop, and program methods to consolidate and analyze structured/unstructured diverse sources, generating actionable insights and solutions for your client services and product enhancement.

With our expertise in Python, Machine Learning, Statistics, Business Intelligence, Data Analytics, SQL programming, Data Visualization, and Time Series Forecasting, we can help you build products for analysis, interact with product and service teams to identify questions and issues for data analysis and experiments. Our team can develop and code software programs, algorithms, and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources.

At AtliQ, we have a proven track record of empowering